### Steps followed
1. Read data
2. Clean the data
3. Split the data
4. Use different encodings
5. Use different models
6. Using final model predict


In [2]:
pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 10.7 MB/s eta 0:00:00


In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN,LSTM,GRU,Dense,Dropout,Embedding
from tensorflow.keras.callbacks import EarlyStopping


from nltk import sent_tokenize
# from gensim.utils import simple_preprocess

from sklearn.metrics import accuracy_score

In [4]:
#  import required libraries
import pandas as pd
import numpy as np
import os
# import gensim
import re

import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from contractions import fix

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/basavarajbijali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/basavarajbijali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/basavarajbijali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
os.getcwd()

'/content'

In [6]:
os.chdir(r'/content/drive/MyDrive/AI/NLP/4.RNN_LSTM_GRU')

#### Read the data

In [7]:
data_path=r'/content/drive/MyDrive/AI/NLP/Data'
file_name=r'IMDB Dataset.csv'

In [8]:

df=pd.read_csv(os.path.join(data_path,file_name))

In [9]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [10]:
df.shape

(50000, 2)

#### Clean the data

In [11]:
from gensim.utils import simple_preprocess

In [12]:

def preprocess_text(text):
    # Expand contractions[example can't--->> cannot]
    text = fix(text)

    #  remove some functuals
    f=['<br /><br />']
    for i in f:
      text=re.sub(i,'',text)

    # Lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))


    # Tokenization
    words = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # # Stemming (or you could use Lemmatization here)
    # stemmer = PorterStemmer()
    # words = [stemmer.stem(word) for word in words]

    # Rejoin words into a single string
    preprocessed_text = ' '.join(words)

    return preprocessed_text

In [13]:
# def preprocess_text(text):
#   return ' '.join(simple_preprocess(text))

In [14]:
df['cleaned_review']=df['review'].agg(preprocess_text)

<ipython-input-14-b0f17fe0af06>:1: FutureWarning: using <function preprocess_text at 0x7e26c3da6a70> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df['cleaned_review']=df['review'].agg(preprocess_text)


In [15]:
df.head()

,review,sentiment,cleaned_review
0,One of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching oz episode ho...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically family little boy jake thinks zombie...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...


In [16]:
df['sentiment'].value_counts(normalize=True)*100

,proportion
sentiment,
positive,50.0
negative,50.0


In [17]:
df['sentiment_Y']=df['sentiment'].agg(lambda x:1 if x=='positive' else 0)

<ipython-input-17-485a0d4f5207>:1: FutureWarning: using <function <lambda> at 0x7e26c35f4940> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df['sentiment_Y']=df['sentiment'].agg(lambda x:1 if x=='positive' else 0)


In [18]:
df['sentiment_Y'].value_counts(normalize=True)*100

,proportion
sentiment_Y,
1,50.0
0,50.0


#### 3. Split the data

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X,y=df['cleaned_review'],df['sentiment_Y']

In [21]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,stratify=y,random_state=42)

In [22]:
y_train.value_counts(normalize=True)*100

,proportion
sentiment_Y,
1,50.0
0,50.0


In [23]:
y_test.value_counts(normalize=True)*100

,proportion
sentiment_Y,
0,50.0
1,50.0


In [24]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((35000,), (15000,), (35000,), (15000,))

#### Use different Encoding technques

In [21]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [22]:
bow=CountVectorizer(max_features=500)
tfidf=TfidfVectorizer(max_features=500)

In [23]:
bow.fit(x_train)

CountVectorizer(max_features=500)

In [24]:
tfidf.fit(x_train)

TfidfVectorizer(max_features=500)

In [25]:
x_train_tfidf=tfidf.transform(x_train).toarray()


In [26]:
x_test_tfidf=tfidf.transform(x_test).toarray()

In [27]:
x_train_bow=bow.transform(x_train).toarray()
x_test_bow=bow.transform(x_test).toarray()

In [28]:
x_train_tfidf.shape,x_test_tfidf.shape

((35000, 500), (15000, 500))

In [30]:
x_train_bow.shape,x_test_bow.shape

((35000, 500), (15000, 500))

5. Try different models

In [39]:
#  logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [36]:
logreg=LogisticRegression()

In [37]:
logreg.fit(x_train_bow,y_train)

LogisticRegression()

In [38]:
y_test_pred=logreg.predict(x_test_bow)

In [45]:
print(f'Logistic regression test accuracy for BOW embed data is {round(accuracy_score(y_true=y_test,y_pred=y_test_pred),3)}')

Logistic regression test accuracy for BOW embed data is 0.84


In [44]:
#  for tfidf data
logreg=LogisticRegression()
logreg.fit(x_train_tfidf,y_train)
y_test_pred_tfidf=logreg.predict(x_test_tfidf)
print(f'Logistic regression test accuracy for TFIDF embed data is {round(accuracy_score(y_true=y_test,y_pred=y_test_pred_tfidf),3)}')


Logistic regression test accuracy for BOW embed data is 0.839


In [46]:
#  Random forest model
from sklearn.ensemble import RandomForestClassifier
frst=RandomForestClassifier()
frst.fit(x_train_tfidf,y_train)
y_test_pred_tfidf=frst.predict(x_test_tfidf)
print(f'Logistic regression test accuracy for TFIDF embed data is {round(accuracy_score(y_true=y_test,y_pred=y_test_pred_tfidf),3)}')



Logistic regression test accuracy for BOW embed data is 0.815


In [47]:
#  Random forest model
from sklearn.ensemble import RandomForestClassifier
frst=RandomForestClassifier()
frst.fit(x_train_bow,y_train)
y_test_pred_bow=frst.predict(x_test_bow)
print(f'Logistic regression test accuracy for TFIDF embed data is {round(accuracy_score(y_true=y_test,y_pred=y_test_pred_bow),3)}')



Logistic regression test accuracy for TFIDF embed data is 0.813


In [48]:
#  lets try word2vec embedding
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [50]:
df.columns

Index(['review', 'sentiment', 'cleaned_review', 'sentiment_Y'], dtype='object')

In [60]:
story = []
for doc in x_train:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [61]:
story

[['want',
  'see',
  'true',
  'thriller',
  'rent',
  'director',
  'screenwriter',
  'scream',
  'feature',
  'overacting',
  'overpaid',
  'tv',
  'actors',
  'passing',
  'stars',
  'run',
  'mill',
  'special',
  'effects',
  'bonanza',
  'instead',
  'get',
  'topnotch',
  'edgy',
  'strong',
  'violence',
  'yet',
  'thrilling',
  'nailbiter'],
 ['one',
  'first',
  'best',
  'columbos',
  'starring',
  'robert',
  'culp',
  'ray',
  'milland',
  'robert',
  'culp',
  'appeared',
  'another',
  'columbo',
  'several',
  'villains',
  'including',
  'patrick',
  'mcgoohan',
  'william',
  'shatner',
  'jack',
  'cassidy',
  'ray',
  'milland',
  'also',
  'made',
  'later',
  'appearancein',
  'one',
  'ray',
  'milland',
  'convinced',
  'beautiful',
  'wife',
  'played',
  'patricia',
  'crowley',
  'affair',
  'hires',
  'culp',
  'investigate',
  'culp',
  'blackmailing',
  'business',
  'side',
  'gives',
  'milland',
  'fake',
  'report',
  'threatens',
  'crowley',
  'real

In [62]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [63]:
model.build_vocab(story)

In [64]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(18940279, 20197090)

In [65]:
len(model.wv.index_to_key)

66924

In [66]:
model.wv.index_to_key

['movie',
 'film',
 'one',
 'like',
 'would',
 'good',
 'even',
 'time',
 'really',
 'see',
 'story',
 'much',
 'could',
 'well',
 'get',
 'bad',
 'great',
 'also',
 'people',
 'first',
 'movies',
 'made',
 'films',
 'make',
 'way',
 'characters',
 'think',
 'watch',
 'many',
 'seen',
 'character',
 'two',
 'never',
 'love',
 'know',
 'acting',
 'best',
 'little',
 'plot',
 'show',
 'ever',
 'life',
 'better',
 'say',
 'still',
 'scene',
 'end',
 'man',
 'scenes',
 'something',
 'go',
 'back',
 'us',
 'real',
 'watching',
 'thing',
 'actors',
 'years',
 'going',
 'though',
 'actually',
 'funny',
 'makes',
 'find',
 'look',
 'another',
 'work',
 'nothing',
 'lot',
 'new',
 'every',
 'old',
 'want',
 'part',
 'director',
 'quite',
 'things',
 'pretty',
 'got',
 'cast',
 'around',
 'seems',
 'young',
 'enough',
 'fact',
 'horror',
 'take',
 'give',
 'world',
 'thought',
 'big',
 'may',
 'however',
 'always',
 'without',
 'saw',
 'music',
 'gets',
 'long',
 'must',
 'right',
 'original',
 

In [58]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [67]:
from tqdm import tqdm
x_train_w2v = []

for doc in tqdm(x_train.values):
    x_train_w2v.append(document_vector(doc))





100%|██████████| 35000/35000 [24:38<00:00, 23.68it/s]


In [68]:
x_test_w2v=[]
for doc in tqdm(x_test.values):
    x_test_w2v.append(document_vector(doc))

100%|██████████| 15000/15000 [12:06<00:00, 20.64it/s]


In [69]:
#  Random forest model
from sklearn.ensemble import RandomForestClassifier
frst=RandomForestClassifier()
frst.fit(x_train_w2v,y_train)
y_test_pred_w2v=frst.predict(x_test_w2v)
print(f'Logistic regression test accuracy for TFIDF embed data is {round(accuracy_score(y_true=y_test,y_pred=y_test_pred_w2v),3)}')



Logistic regression test accuracy for TFIDF embed data is 0.836


RNN

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN,LSTM,GRU,Dense,Dropout,Embedding
from tensorflow.keras.callbacks import EarlyStopping


In [26]:
tokenize=Tokenizer()

In [27]:
tokenize.fit_on_texts(x_train)

In [28]:
x_train_sequence = tokenize.texts_to_sequences(x_train)

In [26]:
x_train_sequence

[[73,
  10,
  177,
  623,
  693,
  75,
  2669,
  1862,
  625,
  3575,
  24002,
  129,
  57,
  2582,
  288,
  388,
  5771,
  184,
  170,
  8865,
  185,
  15,
  5006,
  4332,
  433,
  438,
  135,
  2982,
  27341],
 [3,
  20,
  37,
  17354,
  1044,
  460,
  14330,
  1164,
  8372,
  460,
  14330,
  1297,
  66,
  3039,
  306,
  1563,
  426,
  2067,
  29653,
  853,
  6509,
  534,
  7188,
  1164,
  8372,
  18,
  22,
  188,
  67398,
  3,
  1164,
  8372,
  2375,
  191,
  210,
  139,
  4661,
  25583,
  1298,
  5970,
  14330,
  3837,
  14330,
  18820,
  796,
  379,
  270,
  8372,
  997,
  3862,
  5033,
  25583,
  54,
  3,
  834,
  15,
  464,
  399,
  67399,
  220,
  3403,
  1666,
  2040,
  67400,
  161,
  15,
  3039,
  293,
  426,
  3511,
  176,
  3039,
  67401,
  274],
 [3,
  1,
  10,
  67402,
  196,
  3100,
  61,
  139,
  1,
  1504,
  3748,
  14766,
  1878,
  46,
  1636,
  1504,
  1439,
  123,
  1688,
  1617,
  1119,
  15207,
  114,
  67403,
  469,
  7,
  2338,
  6,
  11226,
  34,
  2067,
  608

In [29]:
max_len=max([len(l) for l in x_train_sequence])

In [30]:
max_len

1408

In [31]:
pd.Series([len(l) for l in x_train_sequence]).quantile(0.98)

393.0

In [32]:
word_index_map=tokenize.word_index

In [31]:
word_index_map

{'movie': 1,
 'film': 2,
 'one': 3,
 'like': 4,
 'would': 5,
 'good': 6,
 'even': 7,
 'time': 8,
 'really': 9,
 'see': 10,
 'story': 11,
 'much': 12,
 'could': 13,
 'well': 14,
 'get': 15,
 'bad': 16,
 'great': 17,
 'also': 18,
 'people': 19,
 'first': 20,
 'movies': 21,
 'made': 22,
 'films': 23,
 'make': 24,
 'way': 25,
 'characters': 26,
 'think': 27,
 'watch': 28,
 'many': 29,
 'seen': 30,
 'character': 31,
 'two': 32,
 'never': 33,
 'love': 34,
 'know': 35,
 'acting': 36,
 'best': 37,
 'little': 38,
 'plot': 39,
 'show': 40,
 'ever': 41,
 'life': 42,
 'better': 43,
 'say': 44,
 'still': 45,
 'scene': 46,
 'end': 47,
 'man': 48,
 'scenes': 49,
 'something': 50,
 'go': 51,
 'back': 52,
 'us': 53,
 'real': 54,
 'watching': 55,
 'thing': 56,
 'actors': 57,
 'years': 58,
 'going': 59,
 'though': 60,
 'actually': 61,
 'funny': 62,
 'makes': 63,
 'find': 64,
 'look': 65,
 'another': 66,
 'work': 67,
 'nothing': 68,
 'lot': 69,
 'new': 70,
 'every': 71,
 'old': 72,
 'want': 73,
 'part': 7

In [33]:
total_words=len(word_index_map)+1

In [135]:
x_train.shape

(35000,)

In [34]:
pad_lenght=400
def prepare_data_for_embedding(x_train,x_test,pad_lenght):
  tokenize=Tokenizer()
  tokenize.fit_on_texts(x_train)
  x_train_sequence = tokenize.texts_to_sequences(x_train)
  x_test_sequence=tokenize.texts_to_sequences(x_test)

  #  pading
  x_train_sequence_pad=pad_sequences(sequences=x_train_sequence,maxlen=pad_lenght,padding='pre')
  x_test_sequence_pad=pad_sequences(sequences=x_test_sequence,maxlen=pad_lenght,padding='pre')

  return (x_train_sequence_pad,x_test_sequence_pad)




In [35]:
x_train_sequence_pad,x_test_sequence_pad=prepare_data_for_embedding(x_train,x_test,pad_lenght=pad_lenght)

In [36]:
# initialize model
model=Sequential(

              layers=[

                      Embedding(input_dim=total_words,output_dim=100,input_length=pad_lenght),
                      LSTM(units=100,return_sequences=True),
                      LSTM(units=50),
                      Dense(units=1,activation='sigmoid')
              ]
)

# compile the model
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

#  initialize callbacks
clbk=EarlyStopping(patience=3,restore_best_weights=True,monitor='val_loss')


#  train the model
model.fit(x_train_sequence_pad,y_train,
          validation_data=(x_test_sequence_pad,y_test),
          callbacks=[clbk],
          epochs=15,
          batch_size=100)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/15
350/350 ━━━━━━━━━━━━━━━━━━━━ 29s 54ms/step - accuracy: 0.7670 - loss: 0.4487 - val_accuracy: 0.8909 - val_loss: 0.2790
Epoch 2/15
350/350 ━━━━━━━━━━━━━━━━━━━━ 36s 54ms/step - accuracy: 0.9560 - loss: 0.1290 - val_accuracy: 0.8879 - val_loss: 0.3274
Epoch 3/15
350/350 ━━━━━━━━━━━━━━━━━━━━ 21s 57ms/step - accuracy: 0.9821 - loss: 0.0544 - val_accuracy: 0.8796 - val_loss: 0.3741
Epoch 4/15
350/350 ━━━━━━━━━━━━━━━━━━━━ 21s 58ms/step - accuracy: 0.9900 - loss: 0.0323 - val_accuracy: 0.8774 - val_loss: 0.4764


In [37]:
model.save('IMDB_LSTM.h5')

In [38]:
y_test_lstm_pred=np.round(model.predict(x_test_sequence_pad))

469/469 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step


In [39]:
accuracy_score(y_test,y_test_lstm_pred)

0.8909333333333334

In [40]:
#  save tokenizer
import pickle

# Save the tokenizer to a file
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenize, file)

### Prediction for input text

In [8]:
#  supportive functions


#. clean the text

def preprocess_text(text):
    # Expand contractions[example can't--->> cannot]
    text = fix(text)

    #  remove some functuals
    f=['<br /><br />']
    for i in f:
      text=re.sub(i,'',text)

    # Lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))


    # Tokenization
    words = text.split()

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # # Stemming (or you could use Lemmatization here)
    # stemmer = PorterStemmer()
    # words = [stemmer.stem(word) for word in words]

    # Rejoin words into a single string
    preprocessed_text = ' '.join(words)

    return preprocessed_text



# to load the pickle file
import pickle
def load_the_model(path,file):
    model_path = os.path.join(path, file)
    model = pickle.load(open(model_path, 'rb'))
    return model

#  to load keras model
def load_keras_model(path,file):
  model_path=os.path.join(path,file)
  k_model=tf.keras.models.load_model(model_path)
  return k_model


In [11]:
#  take input
input_text='Movie is worst, its a waste of time'

#  cleant the text
cleaned_text=preprocess_text(input_text)






In [12]:
cleaned_text

'movie worst waste time'

In [14]:
# load tokenizer and model
tokenizer=load_the_model(os.getcwd(),'tokenizer.pkl')
model=load_keras_model(os.getcwd(),'IMDB_LSTM.h5')

ModuleNotFoundError: No module named 'keras.src.legacy'

In [74]:
pred=np.round(model.predict(final_seq_text))





1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step


In [75]:
res='Positive ' if pred==1 else 'Negavtive'

In [76]:
res

'Negavtive'

In [147]:
def predict_movie_sentiment(text):
  # cleaning the text
  cleaned_text=preprocess_text(input_text)


  #  load tokenizer
  tokenizer=load_the_model(os.getcwd(),'tokenizer.pkl')


  #  sequencing and padding
  seq_text=tokenizer.texts_to_sequences([cleaned_text])

  # padding
  if len(seq_text)>pad_lenght:
    final_seq_text=seq_text[-pad_lenght:]
  else:
    final_seq_text=pad_sequences(seq_text,maxlen=pad_lenght,padding='pre')

  # load model
  model=load_keras_model(os.getcwd(),'IMDB_LSTM.h5')

  #  predict from the model
  pred=model.predict(final_seq_text)

  # getting result
  res='Positive' if pred[0][0]>0.5 else 'Negavtive'

  return res,pred






In [151]:
input_text='Movie is worst, its a waste of time'

In [152]:

res,pred=predict_movie_sentiment(input_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step


In [153]:
res,pred

('Negavtive', array([[0.06672249]], dtype=float32))

In [3]:
file_name=r'https://drive.google.com/file/d/111bbMunryjcyHksERH9v2Tpe08AGENsz/view?usp=drive_link'
file_name=r'https://drive.google.com/file/d/111bbMunryjcyHksERH9v2Tpe08AGENsz/view?usp=sharing'



In [4]:
import streamlit as st
import gdown
import os

# Google Drive file ID
file_id = "111bbMunryjcyHksERH9v2Tpe08AGENsz"

# Destination path
destination = "your_model_name.h5"

# Download the file from Google Drive if it doesn't exist
if not os.path.exists(destination):
    url = f"https://drive.google.com/uc?id={file_id}"
    gdown.download(url, destination, quiet=False)

# Load your model here
# model = load_model(destination)

# st.write("Model downloaded successfully!")


2024-08-27 13:11:25.061 
  command:

    streamlit run /Users/basavarajbijali/anaconda3/envs/DLCVNLP/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [5]:
from keras.models import load_model

In [6]:
import tensorflow as tf

In [7]:
tf.__version__ 

'2.17.0'

In [8]:
model = tf.keras.models.load_model('your_model_name.h5')

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (100, 400, 100)        │    17,001,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (100, 400, 100)        │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (100, 50)              │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (100, 1)               │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,112,153 (65.28 MB)

 Trainable params: 17,112,151 (65.28 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)